In [1]:
import pandas as pd
import os

In [2]:
import glob

def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).__next__()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:
                if "Bibliography" in file:
                    continue
                if("doclist" in file):
                    continue
                if(".docx" in file):
                    r.append(os.path.join(subdir, file))                                                                         
    return r                                                                                                          



In [3]:
file_list = list_files("/Users/Mihir/Downloads/Sample/")

In [4]:
import docx
import pandas as pd
words = []
for i in file_list:
    doc = docx.Document(i)
    words.append([p.text for p in doc.paragraphs])
xml = "/Users/Mihir/Downloads/Sample/wsj.xlsx"
for i in range(len(words)):
    words[i] = ' '.join(words[i])
    words[i] = words[i][words[0].index("Body"):]

In [7]:
from dateutil.parser import parse
import DataFra z
import re
one = []
two = [] 
three = []
for i in words:
    date =parse(i[i.index("Load-Date:") + 11:i.index("End of Document")])
    if(date.year<=2012):
        one.append(i)
    elif(date.year <= 2016):
        two.append(i)
    else:
        three.append(i)
df_one = DataFrame(one, columns = ['text'])
df_two = DataFrame(two, columns = ['text'])
df_three = DataFrame(three, columns = ['text'])
df_one['processed'] = df_one['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_one['processed'] = df_one['text'].map(lambda x: x.lower())

df_two['processed'] = df_two['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_two['processed'] = df_two['text'].map(lambda x: x.lower())

df_three['processed'] = df_three['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_three['processed'] = df_three['text'].map(lambda x: x.lower())

In [26]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = df_three.processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['usa', 'today', 'may', 'wednesday', 'first', 'edition', 'copyright', 'gannett', 'company', 'inc', 'all', 'rights', 'reserved', 'section', 'money', 'pg', 'length', 'words', 'byline', 'jefferson', 'graham', 'jeffersongraham', 'usa', 'today', 'body', 'drone', 'controlled', 'by', 'hand', 'gestures']


In [27]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = " ".join(sent)
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
import nltk
words = []
for i in range(len(data_words_bigrams)):
    wn = nltk.WordNetLemmatizer()
    words.append([wn.lemmatize(word) for word in data_words_bigrams[i]])

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
import re
import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem.porter import PorterStemmer

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
from nltk.corpus import wordnet as wn
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}
verbs = {x.name().split('.', 1)[0] for x in wn.all_synsets('v')}

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
from nltk.corpus import wordnet
dictionary = {}
for i in range(len(words)):
    s = set(words[i])
    for w in s:
        if(w in dictionary):
            dictionary[w]+=1
        else:
            dictionary[w] = 0
for i in range(len(words)):
    temp = []
    for w in words[i]:
        temp.append(w)
    for w in words[i]:
        if(dictionary[w]>=(len(words) * 0.85) or len(w)<=2 or (w not in nouns)):
            temp.remove(w)
    words[i] = temp

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora

id2word = corpora.Dictionary(words)
texts = words
corpus = [id2word.doc2bow(text) for text in words]

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
from pprint import pprint
# number of topics
num_topics = 6
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.009*"company" + 0.008*"military" + 0.006*"technology" + 0.006*"one" + '
  '0.005*"government" + 0.005*"official" + 0.005*"time" + 0.004*"american" + '
  '0.004*"china" + 0.004*"year"'),
 (1,
  '0.008*"military" + 0.007*"american" + 0.007*"one" + 0.006*"official" + '
  '0.006*"time" + 0.005*"company" + 0.005*"year" + 0.005*"syria" + '
  '0.004*"technology" + 0.004*"attack"'),
 (2,
  '0.008*"company" + 0.008*"military" + 0.006*"technology" + 0.006*"israel" + '
  '0.006*"official" + 0.006*"one" + 0.005*"government" + 0.005*"iran" + '
  '0.004*"time" + 0.004*"people"'),
 (3,
  '0.009*"company" + 0.006*"one" + 0.006*"technology" + 0.006*"say" + '
  '0.005*"military" + 0.005*"work" + 0.005*"year" + 0.005*"two" + 0.004*"war" '
  '+ 0.004*"first"'),
 (4,
  '0.007*"military" + 0.007*"company" + 0.005*"one" + 0.005*"government" + '
  '0.005*"technology" + 0.004*"airport" + 0.004*"aircraft" + 0.004*"official" '
  '+ 0.004*"first" + 0.004*"syria"'),
 (5,
  '0.009*"military" + 0.008*"one"

In [37]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/Users/Mihir/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/Users/Mihir/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.017923  0.003263       1        1  23.955287
0      0.002844  0.001958       2        1  21.284180
5     -0.001267 -0.000208       3        1  16.192119
1      0.013260 -0.002906       4        1  15.722739
2      0.005069  0.012341       5        1  13.103586
4     -0.001982 -0.014449       6        1   9.742088, topic_info=          Term        Freq       Total Category  logprob  loglift
1813    israel  209.000000  209.000000  Default  30.0000  30.0000
302   military  529.000000  529.000000  Default  29.0000  29.0000
887       iran  198.000000  198.000000  Default  28.0000  28.0000
147   american  271.000000  271.000000  Default  27.0000  27.0000
1162   israeli  132.000000  132.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
739     attack   21.960154  219.327194   Topic6  -5.7388   0.0274
130       time   24.834630  300.289062   Topic6  -5.6158  -0.1638
1422     china   19.983984  191.740404   Topic6  -5.8331   0.0675
147   american   21.295048  271.270642   Topic6  -5.7696  -0.2159
74        like   20.656720  249.136974   Topic6  -5.8000  -0.1612

[509 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1588      1  0.385174      aaron
1588      2  0.096293      aaron
1588      3  0.224685      aaron
1588      4  0.096293      aaron
1588      5  0.128391      aaron
...     ...       ...        ...
872       5  0.591620  yorkshire
4449      1  0.220025       yuan
4449      2  0.440050       yuan
4449      3  0.220025       yuan
4449      5  0.220025       yuan

[1385 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 6, 2, 3, 5])

In [38]:
df = LDAvis_prepared[1]
df = df.drop(columns = ["Freq", "Total", "logprob", "loglift"])
print(df)
compression_opts = dict(method='zip',
                         archive_name='out.csv')  
df.to_csv('out.zip', index=False,
           compression=compression_opts)  

          Term Category
1813    israel  Default
302   military  Default
887       iran  Default
147   american  Default
1162   israeli  Default
...        ...      ...
739     attack   Topic6
130       time   Topic6
1422     china   Topic6
147   american   Topic6
74        like   Topic6

[509 rows x 2 columns]


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
